In [13]:
import fastf1 as ff1
from fastf1.core import Laps

In [14]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import cm

import numpy as np
import pandas as pd

In [15]:
ff1.Cache.enable_cache("C:/Users/lmbfr/OneDrive/Documents/Travail ESILV/Cours + Exercices/Semestre 7/Python Data Analysis/F1")

In [ ]:
#Grand prix with sprint qualifying (SQ session)
#Silverstone n°10 - Monza n°14 - Interlagos n°19

## Retrieve fastest lap for every practice session + row content overview

In [145]:
def GetPositionsDataframe(year, gpNumber, sessionLetters):
    
    session = None
    
    weekend = ff1.get_session(year, gpNumber)
    if (sessionLetters[0] == "P"):
        session = weekend.get_practice(sessionLetters[1])
        
    laps = session.load_laps()
    drivers = pd.unique(laps['Driver'])
    
    list_fastest_laps = list()
    for drv in drivers:
        drvs_fastest_lap = laps.pick_driver(drv).pick_fastest()
        list_fastest_laps.append(drvs_fastest_lap)
    fastest_laps = Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
    
    df = pd.DataFrame(fastest_laps)
    df['Position'] = np.arange(1, df.shape[0] + 1)
    df['Weekend'] = session
    df['WeekendNumber'] = gpNumber
    return df

In [134]:
fastests = GetPositionsDataframe(2021, 21, 'P1')
fastests.head()

core           INFO 	Loading laps for Saudi Arabian Grand Prix - Practice 1 [v2.1.12]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']


,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,Position,Weekend,WeekendNumber
0,0 days 00:57:56.937000,44,0 days 00:01:29.786000,14,3,NaT,NaT,0 days 00:00:32.907000,0 days 00:00:28.660000,0 days 00:00:28.219000,...,5.0,False,0 days 00:56:27.151000,Mercedes,HAM,1,True,1,<fastf1.core.Session object at 0x00000199D4B71...,21
1,0 days 01:13:01.705000,33,0 days 00:01:29.842000,20,5,NaT,NaT,0 days 00:00:32.765000,0 days 00:00:28.742000,0 days 00:00:28.335000,...,9.0,False,0 days 01:11:31.863000,Red Bull,VER,1,True,2,<fastf1.core.Session object at 0x00000199D4B71...,21
2,0 days 00:52:54.908000,77,0 days 00:01:30.009000,14,2,NaT,NaT,0 days 00:00:32.847000,0 days 00:00:28.791000,0 days 00:00:28.371000,...,6.0,True,0 days 00:51:24.899000,Mercedes,BOT,2,True,3,<fastf1.core.Session object at 0x00000199D4B71...,21
3,0 days 00:57:11.680000,10,0 days 00:01:30.263000,19,3,NaT,NaT,0 days 00:00:32.945000,0 days 00:00:28.821000,0 days 00:00:28.497000,...,8.0,False,0 days 00:55:41.417000,AlphaTauri,GAS,1,True,4,<fastf1.core.Session object at 0x00000199D4B71...,21
4,0 days 00:59:07.815000,99,0 days 00:01:30.318000,19,3,NaT,NaT,0 days 00:00:32.962000,0 days 00:00:28.879000,0 days 00:00:28.477000,...,9.0,True,0 days 00:57:37.497000,Alfa Romeo,GIO,1,True,5,<fastf1.core.Session object at 0x00000199D4B71...,21


In [136]:
fastests.columns

Index(['Time', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime',
       'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'Compound', 'TyreLife',
       'FreshTyre', 'LapStartTime', 'Team', 'Driver', 'TrackStatus',
       'IsAccurate', 'Position', 'Weekend', 'WeekendNumber'],
      dtype='object')

In [137]:
fastests.iloc[0]

Time                                             0 days 00:57:56.937000
DriverNumber                                                         44
LapTime                                          0 days 00:01:29.786000
LapNumber                                                            14
Stint                                                                 3
PitOutTime                                                          NaT
PitInTime                                                           NaT
Sector1Time                                      0 days 00:00:32.907000
Sector2Time                                      0 days 00:00:28.660000
Sector3Time                                      0 days 00:00:28.219000
Sector1SessionTime                               0 days 00:57:00.058000
Sector2SessionTime                               0 days 00:57:28.718000
Sector3SessionTime                               0 days 00:57:56.937000
SpeedI1                                                         

## Data format to fit into quali and race dataframe

In [138]:
#From df_global.columns
lnames = ['number', 'position', 'positionText', 'points', 'grid', 'laps',
       'status', 'Q1', 'Q2', 'Q3', 'year', 'gpName', 'gpNumber', 'sessionName', 'driverId',
       'code', 'DriverNationality', 'constructorId', 'constructorNationality',
       'fastestLapNumber', 'fastestLapRank', 'fastestLapAvgSpeed',
       'fastestLapTime', 'totalTime', 'TimeInterval']

#NaN values
NaNs = [np.NaN for x in range(len(lnames))]

In [139]:
def DataFramePracticeResult(fastests):
    rows = []
    for i in range(fastests.shape[0]):
        row = dict(zip(lnames, NaNs))
        row["number"] = fastests.iloc[i]["DriverNumber"]
        row["position"] = fastests.iloc[i]["Position"]

        row["year"] = fastests.iloc[i]["Weekend"].weekend.year
        row["gpName"] = fastests.iloc[i]["Weekend"].weekend.name
        row["gpNumber"] = fastests.iloc[i]["WeekendNumber"]
        row["sessionName"] = fastests.iloc[i]["Weekend"].name

        #Le driverId et les autres infos sur l'écurie ne sont pas fournies par fastest laps

        row["code"] = fastests.iloc[i]["Driver"]

        #...

        row["fastestLapNumber"] = fastests.iloc[i]["LapNumber"]
        row["fastestLapRank"] = fastests.iloc[i]["Position"]
        #Fastest lap avg. speed needs a calculation
        row["fastestLapTime"] = fastests.iloc[i]["LapTime"]
        
        rows.append(row)
    
    return pd.DataFrame(rows)

## Concatenate all practices records and write it into a .csv file

In [140]:
def WriteAllData():
    session_letter = ['P1', 'P2', 'P3']
    firstrows = True
    
    practiceData = None
    
    for year in range(2014,2022):
        for gp in range(1,23):
            for l in session_letter:
                try:
                    fastests = GetPositionsDataframe(year, gp, l)
                    if (firstrows == True):
                        practiceData = DataFramePracticeResult(fastests)
                        firstrows = False
                    else:
                        practiceData = pd.concat([practiceData, DataFramePracticeResult(fastests)])
                    
                except:
                    print("problem on",year, gp, l)
                    
    practiceData.to_csv(r'C:\Users\lmbfr\OneDrive\Documents\Travail ESILV\Cours + Exercices\Semestre 7\Python Data Analysis\F1\practices.csv', index=False)

In [144]:
WriteAllData()

core           INFO 	Loading laps for Australian Grand Prix - Practice 1 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
core           INFO 	Loading laps for Australian Grand Prix - Practice 2 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
core           INFO 	Loading laps for Australian Grand Prix - Practice 3 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...


problem on 2014 1 P1
problem on 2014 1 P2
problem on 2014 1 P3

core           INFO 	Loading laps for Malaysian Grand Prix - Practice 1 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
core           INFO 	Loading laps for Malaysian Grand Prix - Practice 2 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...



problem on 2014 2 P1


core           INFO 	Loading laps for Malaysian Grand Prix - Practice 3 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
core           INFO 	Loading laps for Bahrain Grand Prix - Practice 1 [v2.1.12]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...


...

problem on 2021 22 P1


api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
core           INFO 	Loading laps for Abu Dhabi Grand Prix - Practice 3 [v2.1.12]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data


problem on 2021 22 P2


api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']


problem on 2021 22 P3
